# Function Calling & Custom Tools

Understanding differences and how to use them together

In [1]:
import pandas as pd
import numpy as np
import json, os
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool

StreamlitPatcher().jupyter()

In [2]:
os.environ["OPENAI_API_KEY"] = ""

### Converting a custom python function to an OpenAI function calling tool

Requires specifying type of arguments, as well as description of function and each arguments. This will be passed to convert_to_openai_tool()

In [3]:
def create_matrix(r:int, c:int):
    """Create a matrix of rows(r) by columns(c)

    Args:
        r: First dimension of matrix
        c: Second dimension of matrix
    """

    m = np.random.randn(r, c)

    return m

convert_to_openai_tool(create_matrix)

{'type': 'function',
 'function': {'name': 'create_matrix',
  'description': 'Create a matrix of rows(r) by columns(c)',
  'parameters': {'type': 'object',
   'properties': {'r': {'type': 'integer',
     'description': 'First dimension of matrix'},
    'c': {'type': 'integer', 'description': 'Second dimension of matrix'}},
   'required': ['r', 'c']}}}

In [10]:

print(json.dumps(convert_to_openai_tool(create_matrix), indent=2))

{
  "type": "function",
  "function": {
    "name": "create_matrix",
    "description": "Create a matrix of rows(r) by columns(c)",
    "parameters": {
      "type": "object",
      "properties": {
        "r": {
          "type": "integer",
          "description": "First dimension of matrix"
        },
        "c": {
          "type": "integer",
          "description": "Second dimension of matrix"
        }
      },
      "required": [
        "r",
        "c"
      ]
    }
  }
}


### Testing ("Binding function) custom function call built in an LLM

We'll incorporate the function call as part of our model so it is "bound" (**binding function**) and ready to be utilized

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

Test it without function calling. LLM will produce actual answer

In [17]:
llm_result = llm.invoke("can you create a random 3x5 matrix?")
llm_result

AIMessage(content='Sure, here is a random 3x5 matrix:\n\n[[4, 7, 2, 9, 1],\n [3, 8, 5, 6, 2],\n [1, 9, 4, 7, 3]]')

Notice that answer will be produced, but it will be produced as part of text

In [19]:
llm_result.content

'Sure, here is a random 3x5 matrix:\n\n[[4, 7, 2, 9, 1],\n [3, 8, 5, 6, 2],\n [1, 9, 4, 7, 3]]'

LLM with function calling will produce json input to pre-specified function ("create_matrix()" in function call)

In [21]:
llm_result_fc = llm.invoke("can you create a random 3x5 matrix?", 
                        tools = [convert_to_openai_tool(create_matrix)])

llm_result_fc

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hSJ0FnN67g7Itf0A3ZkLfyb8', 'function': {'arguments': '{"r":3,"c":5}', 'name': 'create_matrix'}, 'type': 'function'}]})

Notice that content will be null, but this will be report json input ready to be passed to actual function

In [23]:
llm_result_fc.content

''

In [64]:
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["name"])
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["arguments"])

{"r":3,"c":5}
create_pandas


In the above example the regular llm was able to produce an answer for a simple request, but what would happen with a more complex request?

In [52]:
llm_result = llm.invoke("can you create a pandas dataframe of 3x5 with random numbers?")
llm_result

AIMessage(content="Sure! Here is an example code to create a pandas dataframe of size 3x5 with random numbers:\n\n```python\nimport pandas as pd\nimport numpy as np\n\ndata = np.random.randint(0, 100, size=(3, 5))\ndf = pd.DataFrame(data, columns=['A', 'B', 'C', 'D', 'E'])\n\nprint(df)\n```\n\nThis code will create a pandas dataframe with 3 rows and 5 columns filled with random numbers between 0 and 100.")

In [54]:
print(llm_result.content)

Sure! Here is an example code to create a pandas dataframe of size 3x5 with random numbers:

```python
import pandas as pd
import numpy as np

data = np.random.randint(0, 100, size=(3, 5))
df = pd.DataFrame(data, columns=['A', 'B', 'C', 'D', 'E'])

print(df)
```

This code will create a pandas dataframe with 3 rows and 5 columns filled with random numbers between 0 and 100.


In [58]:
def create_pandas(r:int, c:int):
    """Create a pandas dataframe of random numbers of rows(r) by columns(c)

    Args:
        r: First dimension of pandas dataframe
        c: Second dimension of pandas dataframe
    """

    import numpy as np
    import pandas as pd

    m  = np.random.randn(r, c)
    df = pd.DataFrame(m, columns=["a", "b", "c", "d", "e"])

    return df

print(json.dumps(convert_to_openai_tool(create_pandas), indent=2))

{
  "type": "function",
  "function": {
    "name": "create_pandas",
    "description": "Create a pandas dataframe of random numbers of rows(r) by columns(c)",
    "parameters": {
      "type": "object",
      "properties": {
        "r": {
          "type": "integer",
          "description": "First dimension of pandas dataframe"
        },
        "c": {
          "type": "integer",
          "description": "Second dimension of pandas dataframe"
        }
      },
      "required": [
        "r",
        "c"
      ]
    }
  }
}


In [59]:
llm_result_fc = llm.invoke("can you create a pandas dataframe of 3x5 with random numbers?", 
                        tools = [convert_to_openai_tool(create_pandas)])

llm_result_fc

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ecz0zRIuBdKKSunS9rPGAcsI', 'function': {'arguments': '{"r":3,"c":5}', 'name': 'create_pandas'}, 'type': 'function'}]})

In [65]:
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["name"])
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["arguments"])

create_pandas
{"r":3,"c":5}


OK....how about something much more complex, like a figure?

In [66]:
llm_result = llm.invoke("can you create a streamlit linechart showing 2 lines with random numbers going from 0 to 100?")
llm_result

AIMessage(content="Sure! Here is an example code to create a Streamlit line chart with 2 lines showing random numbers going from 0 to 100:\n\n```python\nimport streamlit as st\nimport pandas as pd\nimport numpy as np\n\n# Generate random data\ndata = pd.DataFrame({\n    'x': np.arange(0, 101),\n    'y1': np.random.randint(0, 101, size=101),\n    'y2': np.random.randint(0, 101, size=101)\n})\n\n# Create line chart\nst.line_chart(data[['y1', 'y2']])\n```\n\nYou can run this code in a Python environment with Streamlit installed to see the line chart with 2 lines showing random numbers.")

In [68]:
print(llm_result.content)

Sure! Here is an example code to create a Streamlit line chart with 2 lines showing random numbers going from 0 to 100:

```python
import streamlit as st
import pandas as pd
import numpy as np

# Generate random data
data = pd.DataFrame({
    'x': np.arange(0, 101),
    'y1': np.random.randint(0, 101, size=101),
    'y2': np.random.randint(0, 101, size=101)
})

# Create line chart
st.line_chart(data[['y1', 'y2']])
```

You can run this code in a Python environment with Streamlit installed to see the line chart with 2 lines showing random numbers.


In [70]:
X

array([ 43,  50,  39,  72,  99,  92,  88,  70,  67,  40,  92,  92,  79,
        14,  16,   5,  85,  39,  66,  64,  70,  69,  11,  76,   6,  88,
        83,  56,  41,   3,  26,   4,  27,  58,   5,  81,  74,  44,  51,
        23,   0,  47,   1,  29,  55,  79,  37,  24,  89,  14,  84,  29,
        23,  55,  40,  23,   4,  32,  58,  36,  87,   6,  72,   9,  82,
        44,  88,  51,  18,  27, 100,  25,   7,   4,  10,  96,  56,   0,
        23,  38,  47,  73,  69,  96,  24,  62,  29,  65,  87,  32,  29,
        97,  46,   8,  65,  89,   9,  27,  26,  49,  74])

In [8]:
def create_streamlit_lines(n:int, d:int, l_range:int, h_range:int):
    """Create a streamlit linechart of n number of lines
    of a random distribution of d numbers with the lowest number 
    being higher or equal to l_range
    and the highest number being lower or equal to h_range

    Args:
        n: number of lines
        d: sample size of distribution
        l_range: lowest number of lines random distribution
        h_range: highest number of lines random distribution

    """

    import numpy as np
    import pandas as pd
    import streamlit as st

    df = pd.DataFrame({
        'x': np.arange(0, d),
        'y1': np.random.randint(l_range, h_range, size=d),
        'y2': np.random.randint(l_range, h_range, size=d)
    })

    plot = st.line_chart(df[['y1', 'y2']])

    return(plot)

print(json.dumps(convert_to_openai_tool(create_streamlit_lines), indent=2))

{
  "type": "function",
  "function": {
    "name": "create_streamlit_lines",
    "description": "Create a streamlit linechart of n number of lines\nof a random distribution of d numbers with the lowest number \nbeing higher or equal to l_range\nand the highest number being lower or equal to h_range",
    "parameters": {
      "type": "object",
      "properties": {
        "n": {
          "type": "integer",
          "description": "number of lines"
        },
        "d": {
          "type": "integer",
          "description": "sample size of distribution"
        },
        "l_range": {
          "type": "integer",
          "description": "lowest number of lines random distribution"
        },
        "h_range": {
          "type": "integer",
          "description": "highest number of lines random distribution"
        }
      },
      "required": [
        "n",
        "d",
        "l_range",
        "h_range"
      ]
    }
  }
}


In [9]:
llm_result_fc = llm.invoke("Create a streamlit figure containing 2 lines, each line being of size 10", 
                        tools = [convert_to_openai_tool(create_streamlit_lines)])

llm_result_fc

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BTT5GxtClxWgi3msHs8vSoo3', 'function': {'arguments': '{"n": 2, "d": 10, "l_range": 1, "h_range": 10}', 'name': 'create_streamlit_lines'}, 'type': 'function'}, {'id': 'call_wAHV1JZIrJiNzg8CD31wbZY3', 'function': {'arguments': '{"n": 2, "d": 10, "l_range": 1, "h_range": 10}', 'name': 'create_streamlit_lines'}, 'type': 'function'}]})

In [10]:
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["name"])
print(llm_result_fc.additional_kwargs["tool_calls"][0]["function"]["arguments"])

create_streamlit_lines
{"n": 2, "d": 10, "l_range": 1, "h_range": 10}


As seen the function caller does a pretty good job at it, in numpy, pandas and more complex streamlit function calls. **Even when not having all arguments provided by the user, it makes pretty decent assumptions**

How would we feed this into our function as an agent chain??